In [1]:
%matplotlib inline

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB :
  
  from google.colab import drive
  drive.mount('/content/drive')
  %cd /content/drive/MyDrive/Github

  repository= 'keyword_spotting'
  %cd {repository}
  %ls -a
  # CPU:
  #!pip install pydub torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

  # GPU:
  !pip install pydub torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 -f https://download.pytorch.org/whl/torch_stable.html
    
working_dir=%pwd
if 'PROJET_ESSAI/notebooks' in working_dir :
    print('Wrong working directory')
    %cd ..
    working_dir=%pwd
    print('New directory :',working_dir)

if 'storage' not in locals():   
    storage=dict()

Wrong working directory
/Users/carlos/Documents/LeTaffe/PROJET_ESSAI
New directory : /Users/carlos/Documents/LeTaffe/PROJET_ESSAI


In [2]:
%run main.py --model spect --save_checkpoint 'spect' --num-epochs 1 

imports done
parsing variables...
Using : cpu
Training mode
training loader set up, size 84843
validation loader set up, size 9981
training/validation sets finished in  0:01:16.421632
size of flatten tensor 151552
1  models have been set up
Launching  nameless_exp experience
expérience : 0


  0%|          | 0/1 [00:23<?, ?it/s]


main program finished in  0:00:23.617808


KeyboardInterrupt: 

In [3]:
PATH='mel.pt'
%run readModel.ipynb

Exception: File `'readModel.ipynb.py'` not found.